**主要内容**
- bounding box预测
- IOU、NMS、Anchor box
- YOLO
- region proposals

# 目标定位与特征点检测
- 分类问题：判断图是否为汽车
- 目标定位：判断是否为汽车并定位
- 目标检测：检测不同物体并定位
![com](./Image/com.png)

**目标定位问题**

设计的网络模型结构如下：
![cldet](./Image/cldet.png)

其中输出：存在的对象及定位框

**目标标签**
$$y = \left[ \begin{array}{l}
P_{c}\\\
b_{x}\\\
b_{y}\\\
b_{h}\\\
b_{w}\\\
c_{1}\\\
c_{2}\\\
c_{3}
\end{array} \right]\left| \begin{array}{l}
是否含有对象，0 or 1\\
\\
\\
\\
\\
是否有行人，0 or 1\\
是否有汽车，0 or 1\\
是否有摩托，0 or 1
\end{array} \right.$$
- 当$P_c = 1$时候，表示图片中存在物体
- 当$P_c = 0$时候，不存在物体其他参数没有意义

**损失函数**

若采用平方误差的形式：
- 当$P_c = 1$时：
$$L(\hat y,y)=(\hat y_{1}-y_{1})^{2}+(\hat y_{2}-y_{2})^{2}+\cdots+(\hat y_{8}-y_{8})^{2}$$
- 当$P_c = 0$时：
$$L(\hat y,y)=(\hat y_{1}-y_{1})^{2}$$

在实际使用中，可以定义更好的形式，如：
- 对c1 c2 c3使用对数似然
- 对边界框使用平方误差
- 对$P_c$使用logistic regression损失等

**特征点检测**

由前面的目标定位问题，我们可以知道，神经网络可以通过输出图片上特征点的坐标（x,y），来实现对目标特征的识别和定位标记。
![feature](./Image/feature.png)

如对于人脸表情识别的问题中，我们通过标定训练数据集中特征点的位置信息，来对人脸进行不同位置不同特征的定位和标记。AR的应用就是基于人脸表情识别来设计的，如脸部扭曲、增加头部配饰等。

在人体姿态检测中，同样可以通过对人体不同的特征位置关键点的标注，来记录人体的姿态。

# 目标检测
## 基于滑动窗口的目标检测
**训练模型**

- 训练集X：对所有图片进行剪切，使得物体或非物体都充满整个小图
- 训练集Y：有汽车为1 无汽车标注为0

**检测过程**
- 首先选定一个特定大小的窗口，将窗口内的图片输入到模型中进行预测；
- 以固定步幅滑动该窗口，遍历图像的每个区域，对窗内的各个小图不断输入模型进行预测；
- 继续选取一个更大的窗口，再次遍历图像的每个区域，对区域内是否有车进行预测；
- 遍历整个图像，可以保证在每个位置都能检测到是否有车。

缺点：每个小窗口都要进行卷积计算，计算成本巨大

## 卷积层代替全连接层
对于卷积网络中全连接层，我们可以利用1×11×1大小卷积核的卷积层来替代

![oneone](./Image/oneone.png)
需注意卷积核的个数与隐层神经元个数相同。

## 活动窗口的卷积实现
在我们实现了以卷积层替代全部的全连接层以后，在该基础上进行滑动窗口在卷积层上的操作。下面以一个小的图片为例：
![conv](./Image/conv.png)
我们以上面训练好的模型，输入一个16×16×3大小的整幅图片，图中蓝色部分代表滑动窗口的大小。我们以2为大小的步幅滑动窗口，分别与卷积核进行卷积运算，最后得到4幅10×10×16大小的特征图，然而因为在滑动窗口的操作时，输入部分有大量的重叠，也就是有很多重复的运算，导致在下一层中的特征图值也存在大量的重叠，所以最后得到的第二层激活值（特征图）构成一副12×12×16大小的特征图。对于后面的池化层和全连接层也是同样的过程。

那么由此可知，滑动窗口在整幅图片上进行滑动卷积的操作过程，就等同于在该图片上直接进行卷积运算的过程。所以卷积层实现滑动窗口的这个过程，我们不需要把输入图片分割成四个子集分别执行前向传播，而是把他们作为一张图片输入到卷积神经网络中进行计算，**其中的重叠部分（公共区域）可以共享大量的计算**。

## Bounding Box预测
前面一节的卷积方式实现的滑动窗口算法，使得在预测时计算的效率大大提高。但是其存在的问题是：**不能输出最精准的边界框（Bounding Box）**。

在滑动窗口算法中，我们取的一些**离散的图片子集的位置**，在这种情况下，有可能我们没有得到一个能够完美匹配汽车位置的窗口，也有可能真实汽车的边界框为一个长方形。所以我们需要寻找更加精确的边界框。


# YOLO
yolo可以使得滑动窗口算法找到更精确的边界框
![yolo](./Image/yolo.png)

## yolo过程：
- 在整幅图片上加上较为精细的网格，将图片分割成n×n个小的图片；
- 采用图像分类和定位算法，分别应用在图像的n×n个格子中。
- 定义训练标签：（对于每个网格，定义如前面的向量yi） 
$$y_{i} = \left[ \begin{array}{l}  
P_{c}\  
b_{x}\  
b_{y}\  
b_{h}\  
b_{w}\  
c_{1}\  
c_{2}\  
c_{3}  
\end{array} \right]$$
对于不同的网格 i 有不同的标签向量yi。
- 将n×n个格子标签合并在一起，最终的目标输出Y的大小为：n×n×8（这里8是因为例子中的目标值有8个）

通过这样的训练集训练得到目标探测的卷积网络模型。我们利用训练好的模型，将与模型输入**相同大小**的图片输入到训练好的网络中，得到大小为n×n×8的预测输出。通过观察n×n不同位置的输出值，我们就能知道这些位置中是否存在目标物体，然后也能由存在物体的输出向量得到目标物体的更加精准的边界框。



## YOLO notation:
- 将对象分配到一个格子的过程是：观察对象的中点，将该对象分配到其中点所在的格子中，（即使对象横跨多个格子，也只分配到中点所在的格子中，其他格子记为无该对象，即标记为“0”）；
- YOLO显式地输出边界框，使得其可以具有任意宽高比，并且能输出更精确的坐标，不受滑动窗口算法滑动步幅大小的限制；
- YOLO是一次卷积实现，并不是在n×n网格上进行$n^2$次运算，而是单次卷积实现，算法实现效率高，运行速度快，可以实现实时识别。

## bounding box细节
利用YOLO算法实现目标探测的时候，对于存在目标对象的网格中，定义训练标签Y的时候，边界框的指定参数的不同对其预测精度有很大的影响。这里给出一个较为合理的约定：（其他定值方式可阅读论文）
- 对于每个网格，以左上角为(0,0)，以右下角为(1,1)；
- 中点bx、by表示坐标值，在0~1之间；
- 宽高bh、bw表示比例值，存在>1的情况。

# 交并比（intersection-over-union)
$\rm IoU = \dfrac{交集面积}{并集面积}$

**交并比函数用来评价目标检测算法是否运作良好。**

一般在目标检测任务中，约定如果 IoU⩾0.5，那么就说明检测正确。当然标准越大，则对目标检测算法越严格。得到的IoU值越大越好。

#  非最大值抑制（non-max suppression，NMS）

对于我们前面提到的目标检测算法，可能会对同一个对象做出多次的检测，非最大值抑制可以确保我们的算法对每个对象只检测一次。

**多网格检测同一物体**

**NMS算法思想**
- 在对n×nn×n个网格进行目标检测算法后，每个网格输出的PcPc为一个0~1的值，表示有车的概率大小。其中会有多个网格内存在高概率；
- 得到对同一个对象的多次检测，也就是在一个对象上有多个具有重叠的不同的边界框；
- 非最大值抑制对多种检测结果进行清理：选取最大PcPc的边界框，对所有其他与该边界框具有高交并比或高重叠的边界框进行抑制；
- 逐一审视剩下的边界框，寻找最高的PcPc值边界框，重复上面的步骤。
- 非最大值抑制，也就是说抑制那些不是最大值，却比较接近最大值的边界框。

**NMS算法流程**

以单个对象检测为例：
- 对于图片每个网格预测输出矩阵：yi=[Pc bx by bh bw]，其中Pc表示有对象的概率；
- 抛弃Pc⩽0.6的边界框；
-对剩余的边界框（while）： 
   - 选取最大Pc值的边界框，作为预测输出边界框；
   - 抛弃和选取的边界框IoU⩾0.5的剩余的边界框。
   
对于多对象检测，输出标签中就会有多个分量。正确的做法是：**对每个输出类别分别独立进行一次非最大值抑制**。

# Anchor box
通过上面的各种方法，目前我们的目标检测算法在每个格子上只能检测出一个对象。使用Anchor box 可以同时检测出多个对象。

**重叠目标**
![overlap](./Image/overlap.png)

Anchor box 则是预先定义多个不同形状的Anchor box，我们需要把预测目标对应地和各个Anchor box 关联起来，所以我们重新定义目标向量： 
yi=[Pc bx by bh bw c1 c2 c3 Pc bx by bh bw c1 c2 c3⋯]用这样的多目标向量分别对应不同的Anchor box，从而检测出多个重叠的目标。
- 不使用Anchor box：训练图片中的每个对象，根据对象的中点，分配到对应的格子中。输出大小（例如8）：n×n×8；
- 使用Anchor box：训练图片的每个对象，根据对象的中点，分配到对应的格子中，同时还分配到一个和对象形状的IoU最高的Anchor box 中。输出大小（例如两个Anchor box）：n×n×16。


**难点**：

- 如果我们使用了两个Anchor box，但是同一个格子中却有三个对象的情况，此时只能用一些额外的手段来处理；
- 同一个格子中存在两个对象，但它们的Anchor box 形状相同，此时也需要引入一些专门处理该情况的手段。

**Anchor box的选择**

- 一般人工指定Anchor box 的形状，选择5~10个以覆盖到多种不同的形状，可以涵盖我们想要检测的对象的形状；
- 高级方法：K-means 算法：将不同对象形状进行聚类，用聚类后的结果来选择一组最具代表性的Anchor box，以此来代表我们想要检测对象的形状。

# 完整额YOLO算法目标检测
假设我们要在图片中检测三种目标：行人、汽车和摩托车，同时使用两种不同的Anchor box。

**训练集**

- 输入X：同样大小的完整图片；
- 目标Y：使用3×3网格划分，输出大小3×3×2×8，或者3×3×16
- 对不同格子中的小图，定义目标输出向量Y。

**模型预测**

输入与训练集中相同大小的图片，同时得到每个格子中不同的输出结果：3×3×2×8。

**运行NMS**
- 假设使用了2个Anchor box，那么对于每一个网格，我们都会得到预测输出的2个bounding boxes，其中一个PcP比较高；
- 抛弃概率Pc值低的预测bounding boxes；
- 对每个对象（如行人、汽车、摩托车）分别使用NMS算法得到最终的预测边界框。 

# 候选区域（region proposals）

**R-CNN：**

R-CNN（Regions with convolutional networks），会在我们的图片中选出一些目标的候选区域，从而避免了传统滑动窗口在大量无对象区域的无用运算。

所以在使用了R-CNN后，我们不会再针对每个滑动窗口运算检测算法，而是只选择一些候选区域的窗口，在少数的窗口上运行卷积网络。

具体实现：运用图像分割算法，将图片分割成许多不同颜色的色块，然后在这些色块上放置窗口，将窗口中的内容输入网络，从而减小需要处理的窗口数量。
![region](./Image/region.png)
**改进算法**

- R-CNN：给出候选区域，对每个候选区域进行分类识别，输出对象 标签 和 bounding box，从而在确实存在对象的区域得到更精确的边界框，但速度慢；
- Fast R-CNN：给出候选区域，使用滑动窗口的卷积实现去分类所有的候选区域，但得到候选区的聚类步骤仍然非常慢；
- Faster R-CNN：使用卷积网络给出候选区域。